## Bigram

Bigram are the precursor to transformer models. These are models where only the previous word is used to predict the next word. For example, `"I am a man"` when you are training the `am` it only gets `I` as input. Same with `a` it only gets `am` as input.

Thi is known as the Markov assumption, where the probability of a current word depends only on the previous word.

They are calculated by analyzing a large text corpus and counting the frequency of each bigram. This data is used to estimate the conditional probailites P(word 2| word 1).

In [ ]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [22]:
# hyperparameters
batch_size = 32
block_size = 8
max_iters = 3000 
eval_interval = 300
learning_rate = 1e-2
device = "mps"
eval_iters = 200
n_embd = 384 
n_head = 6
n_layer = 6
dropout = 0.2

In [14]:
with open("./data/karpathy/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [15]:
# char tokenizer
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [16]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [17]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

In [43]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None: 
            loss = None
        else: 
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [47]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [48]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch("train")

    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

In [ ]:
torch.manual_seed(1337)
B, T, C = 4,8,32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

# no communication has happened yet
k = key(x)
q = query(x)

# communication
wei = q @ k.transpose(-2,-1)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
# In encoder blocks you would delete this. Only in decoder blocks would you want the autoregressive format.
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape